# Crawl D&C podcasts from the Church of Jesus Christ of Latter-day Saints

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from urllib.parse import urljoin, urlparse

from bs4 import BeautifulSoup

from models.crawl_utils import get_page, save_page

In [ ]:
# config
base = 'https://doctrineandcovenantscentral.org'
host = 'https://doctrineandcovenantscentral.org/church-history-matters-podcast/'
base_dir = '../data/raw/dc_podcasts'
bs_parser = 'html.parser'
delay_seconds = 5

if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    

In [ ]:
def extract_series_hrefs(html,base):
    hrefs = []

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find all div tags with the class 'views-field-title'
    link_tags = soup.find_all('div', {'data-column-clickable': True})

    # Extract href attribute from each anchor tag and append to the list
    for link in link_tags:
        href = link.get('data-column-clickable')
        if href:
            if href.startswith(base):
                path_components = urlparse(href).path.split('/')
                # print(path_components)
                if len(path_components) == 3:
                    if not href in hrefs:
                        hrefs.append(href)

    return hrefs

In [ ]:
def extract_links(html,base):
    hrefs = []

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find all div tags with the class 'views-field-title'
    link_tags = soup.find_all('div', {'data-column-clickable': True})

    # Extract href attribute from each anchor tag and append to the list
    for link in link_tags:
        href = link.get('data-column-clickable')
        if href:
            if href.startswith(base):
                hrefs.append(href)
                # path_components = urlparse(href).path.split('/')
                # print(path_components)
                # if len(path_components) == 4:
                #    if not href in hrefs:
                #        hrefs.append(href)

    return hrefs

In [ ]:
def get_path(url):
    path_components = urlparse(url).path.split('/')
    # print(path_components)
    return os.path.join(base_dir, f"{path_components[-1]}.json")

In [ ]:
status_code, html = get_page(host, delay_seconds)
if status_code != 200:
    print("Error!", status_code)
series_hrefs = extract_series_hrefs(html,base)
print(series_hrefs[:5])
len(series_hrefs)

In [ ]:
dc_podcasts_pages = []

for series_link in series_hrefs:
    status_code, html = get_page(series_link, delay_seconds)
    if status_code != 200:
        print("Error!", status_code)
    episodes_hrefs = extract_links(html,base)
    print(series_link, len(episodes_hrefs), episodes_hrefs[:3])
    if len(episodes_hrefs) != 0:
        dc_podcasts_pages.extend(episodes_hrefs)
    
print(dc_podcasts_pages[:5])
len(dc_podcasts_pages)

In [ ]:
for url in dc_podcasts_pages:
    if url.endswith('/'):
        url = url[:-1]
    path_file =  get_path(url)
    print(path_file)
    if os.path.exists(path_file):
        continue
    status_code, html = get_page(url, delay_seconds)
    if status_code != 200:
        print("Error!", status_code , url)
        continue
    
    save_page(path_file,url,html)
    
print("End")